In [1]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
import pandas as pd

In [2]:
inference_server_url = "http://0.0.0.0:8000/v1"

chat = ChatOpenAI(
    model="models/llama3_lora_sft_tags/",
    openai_api_key="None",
    openai_api_base=inference_server_url,
    temperature=0,
)

In [6]:
classes = ["คอมพิวเตอร์และเทคโนโลยี", "สุขภาพและการแพทย์", "คณิตศาสตร์และวิทยาศาสตร์", "ทักษะชีวิตและการพัฒนาตนเอง", "ภาษาและการสื่อสาร", "เกษตรและสิ่งแวดล้อม", "การศึกษาและการฝึกอบรม", "วิศวกรรมและสถาปัตยกรรม", "อาหารและโภชนาการ", "ศิลปวัฒนธรรมและศาสนา", "ธุรกิจและการบริหารจัดการ", "สังคม การเมืองการปกครอง", "โครงการมหาวิทยาลัยไซเบอร์ไทย"]
INSTRUCTION = "จงทำนาย Tags สำหรับคอร์สเรียนใน ThaiMOOC (แพลตฟอร์มสื่อการสอนออนไลน์)"
PROMPT = """\
ลักษณะข้อมูล ข้อมูลคอร์สใน ThaiMOOC ในรูปแบบ csv โดยจะมีลักษณะโครงสร้างข้อมูลดังต่อไปนี้:

title : ชื่อรายวิชา
description : คำอธิบายรายวิชา
institute : สถาบันต้นสังกัด
tag : tag ที่จะใช้ทำนาย สามารถมีได้หลาย tag ในหนึ่งคอร์สได้

นี่คือ tag ทั้งหมดที่สามารถตอบได้ !!ห้ามตอบ tag นอกเหนือจากนี้!!:
[{classes}]

เริ่ม!

title : {title}
description : {description}
institute : {institute}
tag :"""

def pred(messages: str):
    messages = [
        SystemMessage(
            content="You are a helpful assistant who're always speak Thai."
        ),
        HumanMessage(
            content=INSTRUCTION + "\n" + messages
        ),
    ]
    return chat(messages).content

In [4]:
test = pd.read_csv("../train_lora/data/test_tags.csv")
test

,index,title,institute,description
0,0,การออกแบบสื่อสามมิติด้วยโปรแกรม Blender 3D | 3...,มหาวิทยาลัยราชภัฏเชียงราย,ศึกษาความรู้พื้นฐานงานออกแบบภาพกราฟิก 3 มิติเพ...
1,1,เทคนิคการถ่ายภาพในแบบสื่อสาธารณะ | Shooting te...,สถานีโทรทัศน์ไทยพีบีเอส (Thai PBS),ศึกษาเทคนิคการถ่ายภาพ การถ่ายวิดีโอในรูปแบบต่า...
2,2,ผู้ประกอบการร้านค้าออนไลน์ | e-Commerce for En...,วิทยาลัยเทคโนโลยีภาคใต้,NaN
3,3,ระบบราชการไทย | The Thai Civil Service System,สำนักงานคณะกรรมการข้าราชการพลเรือน,NaN
4,4,การจัดการทรัพยากรมนุษย์ | Human Resource Manag...,มหาวิทยาลัยนเรศวร,ทุกท่านจะได้เรียนรู้เกี่ยวกับมโนทัศน์การจัดการ...
...,...,...,...,...
173,173,การวิจัยด้านเทคโนโลยีสารสนเทศทางธุรกิจ | Resea...,มหาวิทยาลัยราชภัฏเชียงใหม่,NaN
174,174,โปรแกรมคอมพิวเตอร์และขั้นตอนวิธีทางคอมพิวเตอร์...,มหาวิทยาลัยขอนแก่น,ความรู้พื้นฐานเกี่ยวกับการทำงานของระบบคอมพิวเต...
175,175,การผลิตวรรณกรรมสำหรับเด็กและเยาวชน Generation ...,มหาวิทยาลัยขอนแก่น,ความหมาย ความสำคัญ ประเภทของวรรณกรรมสำหรับเด็ก...
176,176,อินเทอร์เน็ตของสรรพสิ่ง : จากแนวคิดไปสู่ผลิตภั...,มหาวิทยาลัยวลัยลักษณ์,อินเทอร์เน็ตของสรรพสิ่งเบื้องต้นและการประยุกต์...


In [24]:
from tqdm.auto import tqdm

for idx, row in tqdm(test.iterrows(), total=len(test)):
    messages = PROMPT.format(
        classes=", ".join(classes),
        title=row["title"],
        description=row["description"],
        institute=row["institute"]
    )
    tags = pred(messages)
    test.loc[idx, "tag"] = tags

  0%|          | 0/178 [00:00<?, ?it/s]

In [25]:
test["tag"].tolist()

['คอมพิวเตอร์และเทคโนโลยี',
 'คอมพิวเตอร์และเทคโนโลยี',
 'ธุรกิจและการบริหารจัดการ',
 'สังคม การเมืองการปกครอง',
 'ธุรกิจและการบริหารจัดการ',
 'คอมพิวเตอร์และเทคโนโลยี',
 'ธุรกิจและการบริหารจัดการ',
 'ทักษะชีวิตและการพัฒนาตนเอง',
 'ทักษะชีวิตและการพัฒนาตนเอง',
 'สังคม การเมืองการปกครอง',
 'ทักษะชีวิตและการพัฒนาตนเอง',
 'การศึกษาและการฝึกอบรม',
 'สุขภาพและการแพทย์',
 'สังคม การเมืองการปกครอง',
 'ธุรกิจและการบริหารจัดการ',
 'สุขภาพและการแพทย์',
 'เกษตรและสิ่งแวดล้อม',
 'คอมพิวเตอร์และเทคโนโลยี',
 'ธุรกิจและการบริหารจัดการ',
 'คอมพิวเตอร์และเทคโนโลยี',
 'ธุรกิจและการบริหารจัดการ',
 'ธุรกิจและการบริหารจัดการ',
 'ภาษาและการสื่อสาร',
 'เกษตรและสิ่งแวดล้อม',
 'สุขภาพและการแพทย์',
 'ธุรกิจและการบริหารจัดการ',
 'สุขภาพและการแพทย์',
 'สุขภาพและการแพทย์',
 'ภาษาและการสื่อสาร',
 'ศิลปวัฒนธรรมและศาสนา',
 'คณิตศาสตร์และวิทยาศาสตร์',
 'ธุรกิจและการบริหารจัดการ',
 'คอมพิวเตอร์และเทคโนโลยี',
 'คอมพิวเตอร์และเทคโนโลยี',
 'ธุรกิจและการบริหารจัดการ',
 'เกษตรและสิ่งแวดล้อม',
 'คอมพิวเตอร์และเทคโนโลยี',
 'ท

In [26]:
test

,index,title,institute,description,tag
0,0,การออกแบบสื่อสามมิติด้วยโปรแกรม Blender 3D | 3...,มหาวิทยาลัยราชภัฏเชียงราย,ศึกษาความรู้พื้นฐานงานออกแบบภาพกราฟิก 3 มิติเพ...,คอมพิวเตอร์และเทคโนโลยี
1,1,เทคนิคการถ่ายภาพในแบบสื่อสาธารณะ | Shooting te...,สถานีโทรทัศน์ไทยพีบีเอส (Thai PBS),ศึกษาเทคนิคการถ่ายภาพ การถ่ายวิดีโอในรูปแบบต่า...,คอมพิวเตอร์และเทคโนโลยี
2,2,ผู้ประกอบการร้านค้าออนไลน์ | e-Commerce for En...,วิทยาลัยเทคโนโลยีภาคใต้,NaN,ธุรกิจและการบริหารจัดการ
3,3,ระบบราชการไทย | The Thai Civil Service System,สำนักงานคณะกรรมการข้าราชการพลเรือน,NaN,สังคม การเมืองการปกครอง
4,4,การจัดการทรัพยากรมนุษย์ | Human Resource Manag...,มหาวิทยาลัยนเรศวร,ทุกท่านจะได้เรียนรู้เกี่ยวกับมโนทัศน์การจัดการ...,ธุรกิจและการบริหารจัดการ
...,...,...,...,...,...
173,173,การวิจัยด้านเทคโนโลยีสารสนเทศทางธุรกิจ | Resea...,มหาวิทยาลัยราชภัฏเชียงใหม่,NaN,คอมพิวเตอร์และเทคโนโลยี
174,174,โปรแกรมคอมพิวเตอร์และขั้นตอนวิธีทางคอมพิวเตอร์...,มหาวิทยาลัยขอนแก่น,ความรู้พื้นฐานเกี่ยวกับการทำงานของระบบคอมพิวเต...,คอมพิวเตอร์และเทคโนโลยี
175,175,การผลิตวรรณกรรมสำหรับเด็กและเยาวชน Generation ...,มหาวิทยาลัยขอนแก่น,ความหมาย ความสำคัญ ประเภทของวรรณกรรมสำหรับเด็ก...,ภาษาและการสื่อสาร
176,176,อินเทอร์เน็ตของสรรพสิ่ง : จากแนวคิดไปสู่ผลิตภั...,มหาวิทยาลัยวลัยลักษณ์,อินเทอร์เน็ตของสรรพสิ่งเบื้องต้นและการประยุกต์...,คอมพิวเตอร์และเทคโนโลยี


In [27]:
test.drop(["title", "institute", "description"], axis=1).to_csv("submission_tags.csv", index=False)